In [2]:
from llama_index.core.vector_stores.types import MetadataFilters
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)
from llama_index.core.query_engine.multistep_query_engine import MultiStepQueryEngine
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine


def get_perform_query(vector_index, query_content, current_keyword):
    # 更新查询引擎的过滤条件
    value = d[current_keyword]
    index_summary = f"用于回答关于中兴通讯网络运维中{value}的问题"
    step_decompose_transform = StepDecomposeQueryTransform(verbose=True)
    query_engine = vector_index.as_query_engine(
        filters=MetadataFilters(
            filters=[{"key": "product_name",
                      "value": current_keyword}]
        ),
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        similarity_top_k=3
    )
    # hyde = HyDEQueryTransform(include_original=True)
    # query_engine = TransformQueryEngine(query_engine, hyde)

    # MultiStepQuery = MultiStepQueryEngine(
    #      query_engine=query_engine,
    #      query_transform=step_decompose_transform,
    #      index_summary=index_summary,
         
    # )
    # # 执行查询
    # response = MultiStepQuery.query(query_content)
    # if response.response == 'Empty Response':
    
    return query_engine
    
# from llama_index import ServiceContext, VectorStoreIndex, StorageContext
# from llama_index.node_parser import SentenceWindowNodeParser
# from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
# from llama_index.indices.postprocessor import SentenceTransformerRerank
# from llama_index import load_index_from_storage
# import os

# def build_sentence_window_index(
#     document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
# ):
#     # create the sentence window node parser w/ default settings
#     node_parser = SentenceWindowNodeParser.from_defaults(
#         window_size=3,
#         window_metadata_key="window",
#         original_text_metadata_key="original_text",
#     )
#     sentence_context = ServiceContext.from_defaults(
#         llm=llm,
#         embed_model=embed_model,
#         node_parser=node_parser,
#     )
#     if not os.path.exists(save_dir):
#         sentence_index = VectorStoreIndex.from_documents(
#             [document], service_context=sentence_context
#         )
#         sentence_index.storage_context.persist(persist_dir=save_dir)
#     else:
#         sentence_index = load_index_from_storage(
#             StorageContext.from_defaults(persist_dir=save_dir),
#             service_context=sentence_context,
#         )

#     return sentence_index

# def get_sentence_window_query_engine(
#     sentence_index,
#     similarity_top_k=6,
#     rerank_top_n=2,
# ):
#     # define postprocessors
#     postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
#     rerank = SentenceTransformerRerank(
#         top_n=rerank_top_n, model="BAAI/bge-reranker-base"
#     )

#     sentence_window_engine = sentence_index.as_query_engine(
#         similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
#     )
#     return sentence_window_engine

# def build_automerging_index(
#     documents,
#     llm,
#     embed_model="local:BAAI/bge-small-en-v1.5",
#     save_dir="merging_index",
#     chunk_sizes=None,
# ):
#     chunk_sizes = chunk_sizes or [2048, 512, 128]
#     node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
#     nodes = node_parser.get_nodes_from_documents(documents)
#     leaf_nodes = get_leaf_nodes(nodes)
#     merging_context = ServiceContext.from_defaults(
#         llm=llm,
#         embed_model=embed_model,
#     )
#     storage_context = StorageContext.from_defaults()
#     storage_context.docstore.add_documents(nodes)

#     if not os.path.exists(save_dir):
#         automerging_index = VectorStoreIndex(
#             leaf_nodes, storage_context=storage_context, service_context=merging_context
#         )
#         automerging_index.storage_context.persist(persist_dir=save_dir)
#     else:
#         automerging_index = load_index_from_storage(
#             StorageContext.from_defaults(persist_dir=save_dir),
#             service_context=merging_context,
#         )
#     return automerging_index

# from llama_index.node_parser import HierarchicalNodeParser

# from llama_index.node_parser import get_leaf_nodes
# from llama_index import StorageContext
# from llama_index.retrievers import AutoMergingRetriever
# from llama_index.indices.postprocessor import SentenceTransformerRerank
# from llama_index.query_engine import RetrieverQueryEngine


# def get_automerging_query_engine(
#     automerging_index,
#     similarity_top_k=12,
#     rerank_top_n=2,
# ):
#     base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
#     retriever = AutoMergingRetriever(
#         base_retriever, automerging_index.storage_context, verbose=True
#     )
#     rerank = SentenceTransformerRerank(
#         top_n=rerank_top_n, model="BAAI/bge-reranker-base"
#     )
#     auto_merging_engine = RetrieverQueryEngine.from_args(
#         retriever, node_postprocessors=[rerank]
#     )
#     return auto_merging_engine